<a href="https://colab.research.google.com/github/SharovarovAleksandr/Academy/blob/main/Lesson62/Lesson62_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Використайте метод SIFT для витягування ознак з зображень та порівняйте його з методом HOG за допомогою метрик точності.

In [1]:
import cv2
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import numpy as np
from keras.datasets import cifar10
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
import warnings
warnings.filterwarnings("ignore")


if __name__=="__main__": 
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    sift = cv2.xfeatures2d.SIFT_create() #Uses SIFT to get descriptors
    descritores = [] 
   

    #Extract descriptors for all images on the dataset
    for i in np.arange(x_train.shape[0]):
        _,imgDesc = sift.detectAndCompute(x_train[i],None)
        if imgDesc is not None:
            for j in np.arange(imgDesc.shape[0]):
                descritores.append(imgDesc[j])

    descritores = np.array(descritores)
   
    #Apply KMeans to descriptors list to generate 10 centroids corresponding to the 10 visual words
    kmeans = KMeans(n_clusters=10).fit(descritores)

    treino_hist_X=[]
    treino_hist_Y=[]
    
    #For each descriptor of an image, find the closest centroid
    #This return a vector with the number of each centroid for each descriptor of an image
    #Create an histogram out of acho vector for each imagem
    #Associate a label for each histogram
    for i in np.arange(x_train.shape[0]):
         _,imgDesc = sift.detectAndCompute(x_train[i],None)
         if imgDesc is not None:
             kcent = kmeans.predict(imgDesc) 
             hist,_ = np.histogram(kcent,bins=[0,1,2,3,4,5,6,7,8,9,10],normed=True)
             treino_hist_X.append(hist)
             treino_hist_Y.append(y_train[i])


    treino_hist_X = np.array(treino_hist_X).reshape((-1,10))
    treino_hist_Y = np.array(treino_hist_Y).reshape((-1,))


    #Train SVM classifier with histogram and label pairs from the training set
    
    modSVM = SVC()
    modSVM.fit(treino_hist_X,treino_hist_Y.reshape((-1,)))

    matriz_conf_teste = np.zeros((10,10))

    #Aplly SVM on the test set
    #Extract descriptor for each image
    #Find closest centroid
    #Create histogram for each image
    #Predict category with the SVM classifiear
    #Lines correspond to true labels and columns correspond to predictions from the SVM classifier
    preds=[]
    labels=[]
    for i in np.arange(x_test.shape[0]):
         _,imgDesc = sift.detectAndCompute(x_test[i],None)
         if imgDesc is not None:
             kcent = kmeans.predict(imgDesc) 
             hist,_ = np.histogram(kcent,bins=[0,1,2,3,4,5,6,7,8,9,10],normed=True)
             ret = modSVM.predict(hist.reshape((1,-1)))[0]
             preds.append(ret)
             labels.append(y_test[i,0])
             real = y_test[i,0]
             matriz_conf_teste[real,ret] = matriz_conf_teste[real,ret] + 1

   
    print('Accuracy:', accuracy_score(labels,preds))
    print('Precision:', precision_score(labels,preds, average='macro'))
    print('Recall:', recall_score(labels,preds, average='macro'))
    print('F1-score:', f1_score(labels,preds, average='macro'))


170498071/170498071 [==============================] - 4s 0us/step
Accuracy: 0.21932638331996793
Precision: 0.21132440284846762
Recall: 0.2194732779092512
F1-score: 0.20313285284931104
